# COM2004/3004 - Linear Classification

*Copyright &copy; 2023 University of Sheffield. All rights reserved*.

## Objectives

* To gain some practical experience with linear classification with perceptron algorithm.
* To compare the performance of agenerative model and a discriminative model on a challenging classification task.

## 1. Background

In this lab class, we will be practice building a linear classifier using perceptron. We will reuse the abalone data been used in week 5 lab, and you should able to directly comparing the ‘generative modelling’ and ‘discriminative modelling’ ap- proaches to linear classification.

## 2. Introduction

In this notebook we will be reuse data set from the UCI machine-learning repository: abalone data. An abalone is a type of sea snail. The age of a specimen can be determined by cutting the shell through the cone and counting rings through a microscope (rather like trees), but this is a time consuming and expensive procedure. The task here is to try and predict the number of rings given simple external measurements of the weight and dimension of the animal.

For the data set that we are using, the true value for the number of rings is known (i.e., the rings were counted after the snails were measured). Results vary from 1 to 29 rings, so this would usually be treated as a 29-class classification problem. To simplify things a little I have regrouped the data into just two classes of roughly equal size: young (less than 10 rings) and old (10 or more rings). I have also only taken the female samples. There are 7 measurements (which are all quite highly correlated) that are to be used to predict the class label. (The precise meaning of the measurements can be found here http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.names)

Compared to the wine classification task, this task is more challenging. It will be impossible to get 100% correct because the classes are not linearly separable. Further, most of the specimens have either 8, 9, 10 or 11 rings and so lie close to the young/old borderline. However, you should be able to get percentage correct scores that are considerably higher than the 50% that would be expected by guessing alone.

## Task 1. Obtaining data

As in previous lab classes, the data can be read directly into an numpy array using the numpy loadtxt function, as in the cell below. The data will form a matrix with 1,307 rows and 8 columns. Each row is a separate sample (i.e., a different snail). The last column stores a class label (1 for ‘young’ or 2 for ‘old’). Columns 1 to 7 are the results of the 7 length and weight measurements.

In [8]:

import numpy as np
X = np.loadtxt(open("data/abalone.txt", "r"))
X.shape

print(X)
# Same as week 5 lab, we first need to split the data into training and testing.
abalone1 = X[X[:, 0] == 1, :]
abalone2 = X[X[:, 0] == 2, :]
abalone1_test = abalone1[0::2, :]
abalone1_train = abalone1[1::2, :]
abalone2_test = abalone2[0::2, :]
abalone2_train = abalone2[1::2, :]
abalone_test = np.vstack((abalone1_test, abalone2_test))
abalone_test.shape

abalone_train = np.vstack((abalone1_train, abalone2_train))
abalone_train.shape

# Data without the label
abalone_train_data = abalone_train[:, 1:]

nsamples, nfeatures = abalone_train_data.shape

# The orignal label are 1 for positive class and 2 for negative class, it might easier to map labels from (1, 2) to (1, -1)
abalone_train_labels = np.where(abalone_train[:, 0] == 1, 1.0, -1.0)


[[1.     0.53   0.42   ... 0.2565 0.1415 0.21  ]
 [2.     0.53   0.415  ... 0.237  0.1415 0.33  ]
 [2.     0.545  0.425  ... 0.294  0.1495 0.26  ]
 ...
 [1.     0.515  0.4    ... 0.2865 0.123  0.1765]
 [2.     0.565  0.45   ... 0.37   0.239  0.249 ]
 [2.     0.625  0.485  ... 0.531  0.261  0.296 ]]


## 5. Disciminative modeling: the Perceptron learning algorithm ##

 We will look at a discriminative approach to classification in which the

The perceptron learning algorithm is quite easy to implement. The function should take several inputs: training data, training_labels, an initial guess at the weights and a learning rate. **Note, the class labels have to be given the values +1 and -1 for the two classes**. It will return a tuple containing:
<ol>
    <li>learnt w parameters</li>
    <li>the number of iterations performed</li>
    <li>the number of misclassified samples</li>
</ol>

In [9]:
def perce(X, y, w_init, gamma, max_iter=1000):
    """ perce
    A simple but inefficient implementation of the perceptron learning alogorithm

    X - the data matrix. Each row represents a separate sample
    y - a 1-D array of integer class labels corresponding to the rows of X - labels must be +1 or -1
    w_init - the initial weight vector
    gamma - a scalar learning rate
    max_iter - maximum number of iterations (defaults to 1000)
    """
    (N, nfeatures) = X.shape

    # Augment the feature vectors by adding a 1 to each one. (see lecture notes)
    X = np.hstack((X, np.ones((N, 1))))
    nfeatures += 1

    w = w_init  # initialise weights
    iter = 0
    mis_class = N  # start by assuming all samples are misclassified

    while mis_class > 0 and iter < max_iter:
        # SOLUTION
        iter += 1
        mis_class = 0
        gradient = np.zeros(nfeatures)  # initaliase the gradients to 0

        # loop over every training sample.
        for i in range(N):
            # each misclassified point will cause the gradient to change
            if np.inner(X[i, :], w) * y[i] <= 0:
                mis_class += 1
                gradient += -y[i] * X[i, :]
        # update the weight vector ready for the next iteration
        # Note, also that the learning rate decays over time (rho/iter)
        w -= gamma / iter * gradient

    return w, iter, mis_class

Now use perceptron function with the same training data that you used previously in order to learn the weights.

Experiment with different learning rates and different numbers of iterations. The function returns the number of errors that are made on the training set. You want this number to be as low as possible.

In [10]:
gamma = 1000  # Initial learning rate

# Make random initial weights. One weight per feature, plus bias term w_0.
w_init = np.random.normal(0, 1, nfeatures + 1)

# Run the perceptron learning algorithm
w, iter, mis_class = perce(abalone_train_data, abalone_train_labels, w_init, gamma)

percent_correct = 100.0 * (nsamples - mis_class) / nsamples

print(f"{percent_correct:3.2f}")  # Using formatted output string. 2 decimal places.

72.43


**Evaluating the classifier:** You now need to evaluate the $\mathbf{w}$ vector that the learning algorithm has produced. To do this you will need to evaluate $\mathbf{w}’\mathbf{x}+b$ for each element in the $test$ set and then generate a class label by comparing the result against 0, i.e., output 1 or 2 depending on whether the result is higher than or lower than 0. Then compute the percentage of labels that match the correct test set labels. (It is best to write a little function to do this.)

In [11]:
def evaluate_linear_classifier(w, test_data, test_labels):
    """Linear classifier evaluations
    Classification step as in perce_fast

    w - classifier weights
    test_data - the test data matrix
    test_labels - the test labels (+1 or -1)
    """
    (nsamples, nfeatures) = test_data.shape
    test_data = np.hstack((test_data, np.ones((nsamples, 1))))
    mc = (np.dot(test_data, w.transpose()) * test_labels) <= 0
    return np.sum(mc)


# Make test data and test labels
abalone_test = np.vstack((abalone1_test, abalone2_test))
abalone_test_data = abalone_test[:, 1:]
abalone_test_labels = np.where(abalone_test[:, 0] == 1, 1.0, -1.0)
nsamples, nfeatures = abalone_test_data.shape

# Compute the number of misclassifications
mis_class = evaluate_linear_classifier(w, abalone_test_data, abalone_test_labels)

# Convert number of misclassifications into percentage correct
percent_correct = 100.0 * (nsamples - mis_class) / nsamples

print(percent_correct)

72.17125382262996


How well does the new classifier perform? How does performance compare with the linear classifier that you built using Gaussians?